In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

sns.set_theme(style="darkgrid")
current_palette = sns.color_palette()


In [2]:
pd.options.display.min_rows = 20
pd.options.display.max_columns = 500


### Load in the data

In [3]:
## Read in 2022 data from filesystem
df_flights = pd.read_csv("../../data/eda/flights/flight_data_Y2022/flight_data_2022.csv.zip",  low_memory=False)


In [4]:
df_flights = df_flights.rename(columns={"Operating_Airline ": "Operating_Airline", "Flight_Number_Operating_Airline": "Flight_Number"})

In [5]:
airlines = pd.read_csv(
    "../../data/databases/flight_attributes/airlines.csv.zip", low_memory=False)
airline_mapper = airlines.set_index("Code")["Description"]


In [6]:
cancellation_codes = pd.read_csv(
    "../../data/databases/flight_attributes/cancellation_codes.csv", low_memory=False)
cancellation_code_mapper = cancellation_codes.set_index(
    "CANCELLATION_REASON")["CANCELLATION_DESCRIPTION"]


In [7]:
usa_airports = (pd.read_csv("../../data/databases/flight_attributes/usa_airports.csv.zip",
                            usecols= ["iata","latitude","longitude"], 
                            low_memory=False))
usa_airports = usa_airports.rename(columns={"iata": "Origin"})
usa_airports.head()



,Origin,latitude,longitude
0,00M,31.953765,-89.234505
1,00R,30.685861,-95.017928
2,00V,38.945749,-104.569893
3,01G,42.741347,-78.052081
4,01J,30.688012,-81.905944


### Clean the data

Feature Selection

In [10]:
# df_flights[SELECTED_COLUMNS].columns.to_list()


In [8]:
def airline_mapping(df_flights):
    df_flights["Operating_Airline"] = (
        df_flights["Operating_Airline"].map(airline_mapper))
    df_flights["CancellationCode"] = (
        df_flights["CancellationCode"].map(cancellation_code_mapper))
    return df_flights

def tweak_df_flights(df_flights):
    SELECTED_COLUMNS = ["FlightDate", "Operating_Airline", "Tail_Number", "Flight_Number", "OriginAirportID",
                        "Origin", "OriginCityName", "OriginState", "OriginStateName", "DestAirportID", "Dest", "DestCityName",
                        "DestState", "DestStateName", "CRSDepTime", "DepTime", "DepDelay", "DepDel15", "DepDelayMinutes", "DepTimeBlk",
                        "TaxiOut", "WheelsOn", "TaxiIn", "CRSArrTime", "ArrTime", "ArrDelayMinutes", "ArrDel15", "ArrTimeBlk",
                        "Cancelled", "CancellationCode", "Diverted", "CRSElapsedTime", "AirTime", "Distance", "CarrierDelay",
                        "WeatherDelay", "NASDelay", "SecurityDelay", "LateAircraftDelay", "FirstDepTime", "TotalAddGTime",
                        ]
    df_flights = (
        df_flights[SELECTED_COLUMNS]
        .copy()
    )
    df_flights = (pd.merge(df_flights, usa_airports, how="left", on=["Origin"]))
    df_flights.pipe(airline_mapping)
    return df_flights.assign(
        DepTime=df_flights.DepTime.fillna(0).astype("float16"),
        DepDelay=df_flights.DepDelay.fillna(0).astype("float16"),
        DepDel15=df_flights.DepDel15.fillna(0).astype("float16"),
        DepDelayMinutes=df_flights.DepDelayMinutes.fillna(0).astype("float16"),
        TaxiOut=df_flights.TaxiOut.fillna(0).astype("float16"),
        WheelsOn=df_flights.WheelsOn.fillna(0).astype("float16"),
        TaxiIn=df_flights.TaxiIn.fillna(0).astype("float16"),
        ArrDel15=df_flights.ArrDel15.fillna(0).astype("float16"),
        ArrDelayMinutes=df_flights.ArrDelayMinutes.fillna(0).astype("float16"),
        CRSElapsedTime=df_flights.CRSElapsedTime.fillna(0).astype("float16"),
        ArrTime=df_flights.ArrTime.fillna(0).astype("float16"),
        AirTime=df_flights.AirTime.fillna(0).astype("float16"),
        Distance=df_flights.Distance.fillna(0).astype("float16"),
        CarrierDelay=df_flights.CarrierDelay.fillna(0).astype("float16"),
        WeatherDelay=df_flights.WeatherDelay.fillna(0).astype("float16"),
        NASDelay=df_flights.NASDelay.fillna(0).astype("float16"),
        SecurityDelay=df_flights.SecurityDelay.fillna(0).astype("float16"),
        LateAircraftDelay=df_flights.LateAircraftDelay.fillna(
            0).astype("float16"),
        FirstDepTime=df_flights.FirstDepTime.fillna(0).astype("float16"),
        TotalAddGTime=df_flights.TotalAddGTime.fillna(0).astype("float16"),
        CancellationCode=df_flights.CancellationCode.fillna(
            "none").astype("category"),
        Tail_Number=df_flights.Tail_Number.fillna("none").astype("category"),
        FlightDate=pd.to_datetime(df_flights.FlightDate)
    ).astype(
        {
            "Operating_Airline": "category",
            "Origin": "category",
            "Dest": "category",
            "DestState": "category",
            "OriginState": "category",
            "OriginCityName": "category",
            "OriginStateName": "category",
            "DestStateName": "category",
            "DestCityName": "category",
            "DepTimeBlk": "category",
            "ArrTimeBlk": "category",
            "Cancelled": "bool",
            "Diverted": "bool",
            "OriginAirportID": "int16",
            "DestAirportID": "int16",
            "CRSDepTime": "int16",
            "CRSArrTime": "int16",
            "Flight_Number": "int16"
        }
    )


df_flights = tweak_df_flights(df_flights)
df_flights.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 4691967 entries, 0 to 4691966
Data columns (total 43 columns):
 #   Column             Dtype         
---  ------             -----         
 0   FlightDate         datetime64[ns]
 1   Operating_Airline  category      
 2   Tail_Number        category      
 3   Flight_Number      int16         
 4   OriginAirportID    int16         
 5   Origin             category      
 6   OriginCityName     category      
 7   OriginState        category      
 8   OriginStateName    category      
 9   DestAirportID      int16         
 10  Dest               category      
 11  DestCityName       category      
 12  DestState          category      
 13  DestStateName      category      
 14  CRSDepTime         int16         
 15  DepTime            float16       
 16  DepDelay           float16       
 17  DepDel15           float16       
 18  DepDelayMinutes    float16       
 19  DepTimeBlk         category      
 20  TaxiOut            float

In [ ]:
## save to db

In [12]:
df_flights.head()

,FlightDate,Operating_Airline,Tail_Number,Flight_Number,OriginAirportID,Origin,OriginCityName,OriginState,OriginStateName,DestAirportID,Dest,DestCityName,DestState,DestStateName,CRSDepTime,DepTime,DepDelay,DepDel15,DepDelayMinutes,DepTimeBlk,TaxiOut,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelayMinutes,ArrDel15,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,AirTime,Distance,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,latitude,longitude
0,2022-01-06,Delta Air Lines Inc.,N315DN,1581,11697,FLL,"Fort Lauderdale, FL",FL,Florida,12953,LGA,"New York, NY",NY,New York,1126,0.0,0.0,0.0,0.0,1100-1159,0.0,0.0,0.0,1419,0.0,0.0,0.0,1400-1459,True,Airline/Carrier,False,173.0,0.0,1076.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.072583,-80.152750
1,2022-01-06,Delta Air Lines Inc.,N545US,1582,10397,ATL,"Atlanta, GA",GA,Georgia,11697,FLL,"Fort Lauderdale, FL",FL,Florida,1631,1627.0,-4.0,0.0,0.0,1600-1659,15.0,1815.0,5.0,1821,1820.0,0.0,0.0,1800-1859,False,none,False,110.0,93.0,581.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.640444,-84.426944
2,2022-01-06,Delta Air Lines Inc.,N545US,1582,11697,FLL,"Fort Lauderdale, FL",FL,Florida,10397,ATL,"Atlanta, GA",GA,Georgia,1931,1929.0,-2.0,0.0,0.0,1900-1959,13.0,2104.0,10.0,2127,2116.0,0.0,0.0,2100-2159,False,none,False,116.0,83.0,581.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.072583,-80.152750
3,2022-01-06,Delta Air Lines Inc.,N345NB,1583,11697,FLL,"Fort Lauderdale, FL",FL,Florida,14492,RDU,"Raleigh/Durham, NC",NC,North Carolina,1024,1019.0,-5.0,0.0,0.0,1000-1059,17.0,1209.0,3.0,1227,1212.0,0.0,0.0,1200-1259,False,none,False,123.0,93.0,680.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.072583,-80.152750
4,2022-01-06,Delta Air Lines Inc.,N978AT,1584,10397,ATL,"Atlanta, GA",GA,Georgia,12448,JAN,"Jackson/Vicksburg, MS",MS,Mississippi,1117,1113.0,-4.0,0.0,0.0,1100-1159,14.0,1127.0,4.0,1142,1131.0,0.0,0.0,1100-1159,False,none,False,85.0,60.0,341.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.640444,-84.426944


In [ ]:
df_flights.shape


## Statistical Analysis

In [ ]:
df_flights.head(1)


In [ ]:
(df_flights
 .groupby("FlightDate")
 [["DepDelayMinutes"]]
 .mean()
 .plot(title="Distribution of flight delays"))


In [ ]:
(df_flights
 .query("DepDelayMinutes > 0 and DepDelayMinutes < 60")
 [["DepDelayMinutes"]]
 .plot(kind="hist", bins=30, title="Distribution of flight delays within an hour"))
plt.show()


In [ ]:
px.line(df_flights
        .groupby("FlightDate")
        [["DepDelayMinutes"]]
        .mean()
        )


#### Flight Status Categories

Departure Delay Groups

In [ ]:
# Departure Delay intervals until 180 mins
def DepDelay_Class(df_flights):
    df_flights["DepDelayClass"] = None
    df_flights.loc[df_flights["DepDelayMinutes"] <= 15, "DepDelayClass"] = "OnTime"
    df_flights.loc[(df_flights["DepDelayMinutes"] > 15) & (df_flights["DepDelayMinutes"] <= 30), "DepDelayClass"] = "Delay15_30"
    df_flights.loc[(df_flights["DepDelayMinutes"] > 30) & (df_flights["DepDelayMinutes"] <= 45), "DepDelayClass"] = "Delay30_45"
    df_flights.loc[(df_flights["DepDelayMinutes"] > 45) & (df_flights["DepDelayMinutes"] <= 60), "DepDelayClass"] = "Delay45_60"
    df_flights.loc[(df_flights["DepDelayMinutes"] > 60) & (df_flights["DepDelayMinutes"] <= 75), "DepDelayClass"] = "Delay60_75"
    df_flights.loc[(df_flights["DepDelayMinutes"] > 75) & (df_flights["DepDelayMinutes"] <= 90), "DepDelayClass"] = "Delay75_90"
    df_flights.loc[(df_flights["DepDelayMinutes"] > 90) & (df_flights["DepDelayMinutes"] <= 105), "DepDelayClass"] = "Delay90_105"
    df_flights.loc[(df_flights["DepDelayMinutes"] > 105) & (df_flights["DepDelayMinutes"] <= 120), "DepDelayClass"] = "Delay105_120"
    df_flights.loc[(df_flights["DepDelayMinutes"] > 120) & (df_flights["DepDelayMinutes"] <= 135), "DepDelayClass"] = "Delay120_135"
    df_flights.loc[(df_flights["DepDelayMinutes"] > 135) & (df_flights["DepDelayMinutes"] <= 150), "DepDelayClass"] = "Delay135_150"
    df_flights.loc[(df_flights["DepDelayMinutes"] > 150) & (df_flights["DepDelayMinutes"] <= 165), "DepDelayClass"] = "Delay150_165"
    df_flights.loc[(df_flights["DepDelayMinutes"] > 165) & (df_flights["DepDelayMinutes"] <= 180), "DepDelayClass"] = "Delay165_180"
    df_flights.loc[df_flights["Cancelled"], "DepDelayClass"] = "Cancelled"
    
DepDelay_Class(df_flights)

In [ ]:
(df_flights
 .DepDelayClass
 .value_counts(ascending=True)
 .plot(kind='barh', figsize=(10,5), color=current_palette[5], title="Flight Delay Classes 2022")
 )

In [ ]:
df_flights["DepDelayClass"].value_counts()/ df_flights.shape[0]

In [ ]:
# Factors affecting delay
# Assumption: A flight is counted as "on time" if it operated less than 15 minutes later than the
def flight_status(df_flights):
        df_flights["FlightStatus"]="OnTime"

        df_flights.loc[df_flights["DepDel15"] == 0, "FlightStatus"]="OnTime"
        df_flights.loc[(df_flights["DepDelayMinutes"] >= 0) & (df_flights["DepDelayMinutes"] <= 15), "FlightStatus"]="OnTime"
        df_flights.loc[df_flights["WeatherDelay"] > 15, "FlightStatus"]="WeatherDelay"
        df_flights.loc[df_flights["NASDelay"] > 15, "FlightStatus"]="NASDelay"
        df_flights.loc[df_flights["SecurityDelay"] > 15, "FlightStatus"]="SecurityDelay"
        df_flights.loc[df_flights["LateAircraftDelay"] > 15, "FlightStatus"]="LateAircraftDelay"
        df_flights.loc[df_flights["CarrierDelay"] > 15, "FlightStatus"]="CarrierDelay"
        df_flights.loc[df_flights["Cancelled"], "FlightStatus"]="Cancelled"
    

flight_status(df_flights)


In [ ]:
(df_flights
 .FlightStatus
 .value_counts(ascending=True)
 .plot(kind='barh', figsize=(10,5), color=current_palette[1], title="Flight Status 2022")
 )

In [ ]:
df_agg = (df_flights
 .groupby([df_flights["FlightDate"].dt.month, "Operating_Airline"])
 [["FlightStatus"]]
 .value_counts()
 .unstack()
 )
df_agg.style.background_gradient(cmap="Greens")

In [ ]:
#df_flights.loc[df_flights["ArrTime"] == 0]

In [ ]:
(df_flights["FlightStatus"].value_counts())/df_flights.shape[0]

In [ ]:
df_flights["FlightStatus"].value_counts()

In [ ]:
df_flights.head()

In [ ]:
plt.scatter(df_flights["DepDelay"], df_flights["DepDelayMinutes"])

In [ ]:
sns.regplot(x="DepDelay", y="DepDelayMinutes", data=df_flights)
plt.show()

In [ ]:
(df_flights
 [["FlightStatus"]]
 .value_counts()
 .sort_values(ascending=False)
 .to_frame("Row Count")
 )

#### Correlation [Pearson]

In [ ]:
corr = (df_flights
        .select_dtypes('float16')
        .corr())
corr.style.background_gradient(cmap='coolwarm')


In [ ]:
sns.heatmap(
    (df_flights
     .select_dtypes('float16')
     .corr()))
plt.show()


In [ ]:
(df_flights.
 groupby("FlightDate")
 ["Cancelled"]
 .agg("mean")
 .plot()
 )


In [ ]:
(df_flights.
 groupby("FlightDate")
 [["WeatherDelay", "NASDelay"]]
 .agg("mean")
 .plot())


In [ ]:
(df_flights.
 groupby("FlightDate")
 [["CarrierDelay", "LateAircraftDelay"]]
 .agg("mean")
 .plot())


##### Frequency Distribution Table

In [ ]:
(df_flights
 [["Operating_Airline"]]
 .value_counts()
 .sort_values(ascending=False)
 .to_frame("Row Count")
 )


In [ ]:
px.bar(
    df_flights["Operating_Airline"]
    .value_counts()
    .sort_values(ascending=True),
    orientation='h',
    template="plotly_dark",
    title="Flights per airline",
    height=700,
)


In [ ]:
df_flights.head(3)

In [ ]:
(df_flights.
 groupby(["Flight_Number", "Tail_Number", "OriginCityName","DestCityName" ])
[["DepDelayMinutes"]]
#.agg(["mean", "sum"]).head(20)
)


In [ ]:
cols = ["FlightDate","Tail_Number","Flight_Number","Operating_Airline", "OriginCityName","DestCityName", "CRSDepTime", "DepTime","DepDelayMinutes","AirTime", "FlightStatus"]
(df_flights
 .loc[df_flights["DepDelayMinutes"] >10, cols ]
 .sort_values(by=["Flight_Number", "FlightDate", "CRSDepTime"],ascending=[True, True, True]))

In [ ]:
cols = ["FlightDate","Tail_Number","Flight_Number","Operating_Airline", "OriginCityName","DestCityName", "CRSDepTime", "DepTime","DepDelayMinutes","AirTime", "FlightStatus"]
(df_flights[cols]
 .groupby(["Flight_Number", "Operating_Airline"])
 [["DepDelayMinutes"]]
 .mean())